# Action_Clf - Logistic Regression

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Load Data

In [2]:
df = pd.read_parquet(r"input_data\ccd_labeled_train_validate.parquet")

print(df.shape)
display(df.head())                    

(13268, 6)


date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   
3  2024-10-05  2024-10-07_1017--3   
4  2024-10-05  2024-10-07_1017--4   

                                                text product   action  \
0                      i am still waiting on my card    card  inquire   
1  what can i do if my card still hasnt arrived a...    card  inquire   
2  i have been waiting over a week is the card st...    card  inquire   
3  can i track my card while it is in the process...    card  inquire   
4  how do i know if i will get my card or if it i...    card  inquire   

  sentiment  
0  negative  
1  negative  
2  negative  
3   neutral  
4   neutral

In [3]:
# testing - check for duplicates or errors in labels
sorted(df['action'].unique())

['access',
 'activate',
 'apply',
 'block',
 'cancel',
 'close',
 'deposit',
 'dispute',
 'earn',
 'exchange',
 'find',
 'inquire',
 'link',
 'open',
 'pay',
 'receive',
 'redeem',
 'refund',
 'renew',
 'report',
 'reset',
 'retrieve',
 'schedule',
 'select',
 'transfer',
 'unblock',
 'unknown',
 'unlink',
 'update',
 'verify',
 'withdraw']

In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['action'].to_list(), test_size=0.2, random_state=42, stratify=df['action'].to_list())

In [5]:
df['action'].value_counts()

action
inquire     3162
dispute     1009
transfer     662
deposit      644
apply        644
withdraw     570
pay          425
activate     423
report       420
verify       401
unknown      387
link         339
open         338
block        325
update       313
close        285
unblock      264
exchange     239
reset        236
cancel       232
redeem       210
renew        208
access       201
schedule     199
find         185
select       176
retrieve     166
earn         164
refund       151
unlink       147
receive      143
Name: count, dtype: int64

In [6]:
# Preprocess the text data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [7]:
# Train a classifier (Logistic Regression)
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

# Predict X_test

# Error Duplication predicted Labels

In [8]:
# SMake predictions on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
print("Accuracy:", round(accuracy_score(y_test, y_pred), 3))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.574

Classification Report:
               precision    recall  f1-score   support

      access       0.20      0.03      0.04        40
    activate       0.67      0.71      0.69        85
       apply       0.62      0.70      0.65       129
       block       0.41      0.40      0.40        65
      cancel       0.74      0.43      0.55        46
       close       0.58      0.54      0.56        57
     deposit       0.60      0.59      0.59       129
     dispute       0.65      0.71      0.68       202
        earn       0.42      0.15      0.22        33
    exchange       0.32      0.15      0.20        48
        find       0.25      0.05      0.09        37
     inquire       0.56      0.90      0.69       632
        link       0.60      0.53      0.56        68
        open       0.51      0.53      0.52        68
         pay       0.49      0.35      0.41        85
     receive       0.00      0.00      0.00        29
      redeem       0.67      0.29      0

C:\py312_env_IssPlp\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\py312_env_IssPlp\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\py312_env_IssPlp\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Predict a single text

In [9]:
# To make a prediction on new text
def predict_category(text):
    text_tfidf = vectorizer.transform([text])
    prediction = model.predict(text_tfidf)
    return prediction[0]

# Example usage
example_text = "my card is stuck in the atm machine. I want it back."
print("Predicted category:", predict_category(example_text))

Predicted category: withdraw


# Save the Model

In [10]:
# Save the model using joblib
joblib.dump(model, 'trained_model/action_clr_logistic_regression_model.pkl')

print("Model saved!")

Model saved!


# Reload Model and do Predict

In [11]:
del model

In [12]:
def predict_category(text):
    text_tfidf = vectorizer.transform([text])
    prediction = model.predict(text_tfidf)
    return prediction[0]

model = joblib.load('trained_model/action_clr_logistic_regression_model.pkl')

# Example usage
example_text = "Why are your fixed deposit rates so terrible? I don't want to roll over my fixed deposit."
print("Predicted category:", predict_category(example_text))

Predicted category: deposit


# Predict X Test

In [13]:
# Load the model from the file
loaded_model = joblib.load('trained_model/action_clr_logistic_regression_model.pkl')

# Make predictions using the loaded model
predictions = loaded_model.predict(X_test_tfidf)

# # Print the predictions
# print("Predictions:", predictions)

# Evaluate the model
print("Accuracy:", round(accuracy_score(y_test, y_pred),3))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.574

Classification Report:
               precision    recall  f1-score   support

      access       0.20      0.03      0.04        40
    activate       0.67      0.71      0.69        85
       apply       0.62      0.70      0.65       129
       block       0.41      0.40      0.40        65
      cancel       0.74      0.43      0.55        46
       close       0.58      0.54      0.56        57
     deposit       0.60      0.59      0.59       129
     dispute       0.65      0.71      0.68       202
        earn       0.42      0.15      0.22        33
    exchange       0.32      0.15      0.20        48
        find       0.25      0.05      0.09        37
     inquire       0.56      0.90      0.69       632
        link       0.60      0.53      0.56        68
        open       0.51      0.53      0.52        68
         pay       0.49      0.35      0.41        85
     receive       0.00      0.00      0.00        29
      redeem       0.67      0.29      0

C:\py312_env_IssPlp\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\py312_env_IssPlp\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\py312_env_IssPlp\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
